In [0]:
%sql
CREATE OR REPLACE TEMP VIEW fiscal_cal AS
SELECT
    DAY_DT,
    DAY_OF_WK_NUM,
    WK_END_DT,
    CAST(FISC_YR_CD AS INT) AS FISC_YR_CD,
    CAST(CONCAT(FISC_YR_CD, LPAD(FISC_WK_OF_YR_NUM, 2, '0')) AS INT) AS fiscal_week_num,
    FISC_WK_OF_YR_NUM,
    CAST(CONCAT(FISC_YR_CD, LPAD(FISC_PRD_OF_YR_NUM, 2, '0')) AS INT) AS fiscal_period_num,
    FISC_PRD_OF_YR_NUM,
    CAST(CONCAT(FISC_YR_CD, LPAD(FISC_QTR_OF_YR_NUM, 2, '0')) AS INT) AS fiscal_qtr_num,
    FISC_QTR_OF_YR_NUM
FROM main.uc_aa_datamart.fiscal_calendar;


In [0]:
from pyspark.sql.functions import input_file_name, regexp_extract

# Step 1: Define base path (up to FoodLion folder)
base_path = "abfss://fiona@rs05ue2pipadl03.dfs.core.windows.net/FIONA/RDS/Delhaize/InstacartSourceData/FoodLion"


# Step 2: Read matching files from all date folders
df = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(f"{base_path}/*/rpt.8350.*.csv.gz")
    .dropDuplicates()
)

# Step 3: Extract the date from the folder structure
# df_with_date = (
#     df.withColumn("file_path", input_file_name())
#       .withColumn("dt", regexp_extract("file_path", r"FoodLion/(\d{4}-\d{2}-\d{2})/", 1))
# )


# Step 4: Create temporary view
df.createOrReplaceTempView("vw_rpt_8350_foodlion_source")


In [0]:
%sql
select * from vw_rpt_8350_foodlion_source
order by picking_started_date_time_pt desc

In [0]:
%sql
-- create or replace table `test-analytics`.custom_datamart.food_lion_pickrate_merged as (
--        with prism as(select WK_END_DT,
--        IFNULL(sum(units_pck),0) as unitspicked,
--        IFNULL(sum(shft_secs),0) as pickingseconds
-- from `test-analytics`.custom_datamart.pickrate_perf_food_lion
-- group by 1),
-- insta as (select b.wk_end_dt, IFNULL(sum(a.num_picked_items),0) as items,
--        IFNULL(sum(a.picking_time_minutes * 60),0) as picking_seconds from (select * from `test-analytics`.custom_datamart.unata_history UNION SELECT * FROM vw_rpt_8350_foodlion_source) a left join fiscal_cal b on 
--  date(from_utc_timestamp(to_utc_timestamp(picking_started_date_time_pt, 'PST'), 'EST')) = b.DAY_DT
--  group by all),
--  main_date as (select distinct wk_end_dt from (select * from prism union select * from insta)),
--  last (select a.*, (ifnull(b.pickingseconds,0)+ifnull(c.picking_seconds,0)) secs, 
--              (ifnull(b.unitspicked,0)+ifnull(c.items,0)) items from main_date a left join prism b on a.wk_end_dt = b.wk_end_dt
--  left join insta c on a.wk_end_dt = c.wk_end_dt)
-- SELECT 
-- b.fiscal_period_num as ahdz_per,
--        b.fiscal_week_num as ahdz_week,
--        b.fiscal_qtr_num as ahdz_qtr,
--        left(b.fiscal_week_num,4) as year,
-- 'Food Lion' as banner,
--        'PFS' as t2,
--        'Open' as open_closed,
--        '' as date_open,
--        '' as date_closed,
--        b.WK_END_DT as week_end_dt,
--        'GMF' as stor_id,
--        'PFS' as type,
--        0 as total_hrs_without_drivers,
--        0 as total_hrs_with_drivers,
--         IFNULL(sum(items),0) as unitspicked,
--        IFNULL(sum(secs),0) as pickingseconds
--        from last a left join fiscal_cal b on a.wk_end_dt = b.wk_end_dt
--        GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14
-- )

In [0]:
%sql
CREATE OR REPLACE TABLE `test-analytics`.custom_datamart.food_lion_pickrate_merged AS (

  -- Summarize prism data
  WITH prism AS (

    with location as (select distinct StoreId, 
case when P3StoreId is null or P3StoreId = '' then StoreId else P3StoreId end as 
clean_store, 
case when StoreId like 'GM%' then 'PFS' else 'CF' end as type 
from managed_assets.location.storepickuppointmaster_cdm)
select c.week_end_dt WK_END_DT,
      --  b.type,
       sum(a.cnt1_qy) unitspicked,
       sum(timestampdiff(second, a.beg_dt_tm, a.end_dt_tm)) pickingseconds 
from merchandising.pos.emp_perf_fdln_sdm a left join location b on a.dc_id = b.clean_store
left join merchandising.pos.rdt_tm_look_sdm c on date(a.beg_dt_tm) = c.the_date
where b.storeId = 'GMF' and a.task_id = 'P' and a.arg2_cd not in ('R','repick')
group by all
  ),

  -- Summarize insta data from combined sources
  insta AS (
    SELECT 
      b.WK_END_DT,
      IFNULL(SUM(a.num_picked_items), 0) AS items,
      IFNULL(SUM(a.picking_time_minutes * 60), 0) AS picking_seconds
    FROM (
      SELECT * FROM `test-analytics`.custom_datamart.unata_history
      UNION
      SELECT * FROM vw_rpt_8350_foodlion_source
      union
      select * from `test-analytics`.custom_datamart.combined_file_instacart
    ) a
    LEFT JOIN fiscal_cal b 
      ON DATE(FROM_UTC_TIMESTAMP(TO_UTC_TIMESTAMP(picking_started_date_time_pt, 'PST'), 'EST')) = b.DAY_DT
    GROUP BY b.WK_END_DT
  ),

  -- Create master list of week_end_dt
  main_date AS (
    SELECT DISTINCT WK_END_DT FROM prism
    UNION
    SELECT DISTINCT WK_END_DT FROM insta
  ),

  -- Merge the data
  merged_data AS (
    SELECT 
      md.WK_END_DT,
      sum(IFNULL(p.unitspicked, 0) + IFNULL(i.items, 0)) AS items,
      sum(IFNULL(p.pickingseconds, 0) + IFNULL(i.picking_seconds, 0)) AS secs
    FROM main_date md
    LEFT JOIN prism p ON md.WK_END_DT = p.WK_END_DT
    LEFT JOIN insta i ON md.WK_END_DT = i.WK_END_DT
    group by 1)
   SELECT 
    f.fiscal_period_num AS ahdz_per,
    f.fiscal_week_num AS ahdz_week,
    f.fiscal_qtr_num AS ahdz_qtr,
    LEFT(f.fiscal_week_num, 4) AS year,
    'Food Lion' AS banner,
    'PFS' AS t2,
    'Open' AS open_closed,
    '' AS date_open,
    '' AS date_closed,
    f.WK_END_DT AS week_end_dt,
    'GMF' AS stor_id,
    'PFS' AS type,
    0 AS total_hrs_without_drivers,
    0 AS total_hrs_with_drivers,
    IFNULL(SUM(m.items), 0) AS unitspicked,
    IFNULL(SUM(m.secs), 0) AS pickingseconds
  FROM merged_data m
  LEFT JOIN (select distinct wk_end_dt, fiscal_period_num, fiscal_week_num, fiscal_qtr_num from fiscal_cal) f ON m.WK_END_DT = f.WK_END_DT
  GROUP BY 
    all
);

In [0]:
%sql

  -- Summarize prism data
  WITH prism AS (

    with location as (select distinct StoreId, 
case when P3StoreId is null or P3StoreId = '' then StoreId else P3StoreId end as 
clean_store, 
case when StoreId like 'GM%' then 'PFS' else 'CF' end as type 
from managed_assets.location.storepickuppointmaster_cdm)
select c.week_end_dt WK_END_DT,
      --  b.type,
       sum(a.cnt1_qy) unitspicked,
       sum(timestampdiff(second, a.beg_dt_tm, a.end_dt_tm)) pickingseconds 
from merchandising.pos.emp_perf_fdln_sdm a left join location b on a.dc_id = b.clean_store
left join merchandising.pos.rdt_tm_look_sdm c on date(a.beg_dt_tm) = c.the_date
where b.storeId = 'GMF' and a.task_id = 'P' and a.arg2_cd not in ('R','repick')
group by all
  ),

  -- Summarize insta data from combined sources
  insta AS (
    SELECT 
      b.WK_END_DT,
      IFNULL(SUM(a.num_picked_items), 0) AS items,
      IFNULL(SUM(a.picking_time_minutes * 60), 0) AS picking_seconds
    FROM (
      SELECT * FROM `test-analytics`.custom_datamart.unata_history
      UNION
      SELECT * FROM vw_rpt_8350_foodlion_source
    ) a
    LEFT JOIN fiscal_cal b 
      ON DATE(FROM_UTC_TIMESTAMP(TO_UTC_TIMESTAMP(picking_started_date_time_pt, 'PST'), 'EST')) = b.DAY_DT
    GROUP BY b.WK_END_DT
  ),

  -- Create master list of week_end_dt
  main_date AS (
    SELECT DISTINCT WK_END_DT FROM prism
    UNION
    SELECT DISTINCT WK_END_DT FROM insta
  )

  -- Merge the data
    SELECT 
      md.WK_END_DT,
      sum(IFNULL(p.unitspicked, 0)) prism_items,
      sum(IFNULL(p.pickingseconds, 0)) as prism_secs,
      sum(IFNULL(i.items, 0)) AS unata_items,
      round(SUM(IFNULL(i.picking_seconds, 0)),0) AS unata_secs
    FROM main_date md
    LEFT JOIN prism p ON md.WK_END_DT = p.WK_END_DT
    LEFT JOIN insta i ON md.WK_END_DT = i.WK_END_DT
    group by 1